# Neural Network Classifier Python Implementation #

Here I present a python implementation of a Neural Network from scratch. I started from the lectures and (MATLAB) exercises on Neural Networks from Andrew NG course "Machine Learning" available on Coursera.
I then expand the options. In this implementation you can add new layer and units on the Neural Network without the need to modify the code. This is a mere exercise and it is not intended as state of the art. Still the results are pretty satisfying and I have anjoyed a lot doing it. I have tested the algorithm on hand writing recognition data provided by the already mentioned "Machine Learning" Course.

In [203]:
import numpy as np
from scipy.optimize import fmin_cg

def sigmoid(X):
    """
    It returns the sigmoid function applied to the matrix X. X may have any shape.
    """
    sig = 1/(1+np.exp(-X))
    return sig

def sigmoidGradient(X):
    """
    It returns the derivative of the sigmoid function calculated at X. X is a matrix and may have any shape.
    """
    sig_g = sigmoid(X)*(1-sigmoid(X))
    return sig_g

def Rand_param_Theta(In_value, Out_value):
    """
    It returns a (Out_value x In_value+1) random matrix with small entries.
    """
    epsilon_init = 0.12
    Rand_param = np.random.rand(Out_value, In_value+1) * 2 * epsilon_init - epsilon_init
    return Rand_param


def nn_predict(nn_params, layers_data, X, min_class_pred = 0):
    """
    The function returns a vector with the classification prediction for the set X.
    min_class_pred identify the smaller integer in the classification and must be adjusted accordingly to your
    classification labels. For instance if your labels are 1-10 you need to set min_class_pred =1. This is because
    the predictions are obtained from the indeces of a matrix and python as base index 0. By default it is setted
    on zero.
    """
    Theta_vec = reshape_thetas(nn_params, layers_data) 
    m = np.shape(X)[0]
    h_pred = [0 for i in range(len(Theta_vec)+1)]
    h_pred[0] = np.hstack((np.ones(m)[np.newaxis].T, X))
    for i in range(len(h_pred)-1):
        h = sigmoid(h_pred[i]@Theta_vec[i].T)
        if i == len(h_pred)-2:

            h_pred[i+1] = h                                           # h_pred[-1] is (num_exp x num_label) matrix. The entry (i,j) contains the probabilities
                                                                      # that the i-th experiment is classified as label j
        else:
            h_pred[i+1] = np.hstack((np.ones(m)[np.newaxis].T, h))

    return np.argmax(h_pred[-1],axis = 1)+min_class_pred

def reshape_thetas(nn_params,layers_data):
    """
    Given a vector v and a list of integers L of lenght n it will returns a list of n-1 matrices exatrapolated by the vector v.
    The shape of a returned matrix is (j x (i+1)) where [...,i,j,..] are two consecutive entries of the list L.
    If it will not be possible automatically rise an Error. The order to reshape the matrices is F = Fortran.
    """
    T_list = []
    start_ind = 0
    for i in range(len(layers_data)-1):
        n = layers_data[i+1]
        m = layers_data[i] + 1
        dum = np.reshape(nn_params[start_ind:start_ind+n*m], (n, m), order = 'F')
        T_list.append(dum)
        start_ind = start_ind + n*m
    return T_list

def  ForwardProp(nn_params, layers_data, X, Y):
    """
    It compute the Forward Propagation of the Neural Network. It returns 
            -a_vec : a vector containing the computed value of the units for each layer.
                    Therefore a_vec[0] are the input units + bias, while a_vec[-1] are
                    the outputs units without the bias to use in the activation function.

           - z_vec : contains the values of the activation function at each unit of the layer
                     excluded the output ones.

           - Y : is a label Matrix of zeroes and ones. Column j has value 1
                 in i-th position if i is the value of the j-th entries of Y.

           - Theta_vec : is the list of the weights matrices.
    """
                ### UTILITIES ###
    Theta_vec = reshape_thetas(nn_params, layers_data)                 # recover the weights matrices from the weights vector

    m = np.shape(X)[0]                                                 # number of experiments
    min_class = np.unique(Y)[0]                                        # smallest value occurring in the classification
    max_class = np.unique(Y)[-1]                                       # larger alue occurring in the classification
    I = np.repeat([np.arange(min_class, max_class +1)],m,0).T          # Classification Matrix
    Y = Y.reshape(Y.size)                                              # Reshape Y in a row vector in order to perform boolean operation
    Y = np.array(I == Y).astype(int)                                   # Matrix of zeroes and ones of the labels. Column j has value 1 in i-th position if i is the value of the j-th entries of Y. 


    ### FORWARD ROPAGATION ###
    J = 0                                                  # Initialise the cost Function J
    a1 = np.hstack((np.ones(m)[np.newaxis].T, X))          # Add the bias Column at the Input Layer
    a_vec = [0 for i in range(len(layers_data))]           # Initialise the units vector
    z_vec = [0 for i in range(len(layers_data)-1)]         # Initialise the activation function vector
    a_vec[0] = a1.T

    for i in range(len(layers_data)-1):
        zi = Theta_vec[i]@a_vec[i]                  # comupte z(i)
        z_vec[i] = zi
        ai = sigmoid(zi)                            # Values of the unit at layer (i)
        mi = np.shape(ai)[1]
        if i != len(layers_data)-2:
            a_i_bias = np.vstack((np.ones(mi), ai)) # Add the Bias Column at the units
            a_vec[i+1] = a_i_bias
        else:
            a_vec[i+1] = ai

    return a_vec, z_vec, Y,Theta_vec


def cost_func_NN(nn_params, layers_data, X, Y, Lambda =0):
    """
      The function has input:
        - nn params  : a vector of weights parameters
        - layers_data: a vector containing the number of units at each layer (input and output included)
        - X : the training set matrix X
        - Y : the label vector Y
        - J : the regularisation parameter, by default is zero
     The algorithm is vectorised and will return the value of the cost function J on the output layer.
    """

    m = np.shape(X)[0]
    a,_,Y,Theta_vec = ForwardProp(nn_params, layers_data, X, Y)   #computing the units values, the label matrix Y and the weights matrices
    a_out = a[-1]  # the output units of the Neural Network

    reg = (Lambda/(2*m))*(sum([(Theta[:,1:]*Theta[:,1:]).sum() for Theta in Theta_vec])) #regolarisation factor

    J = (1/m) * (((-Y * np.log(a_out))-((1-Y) * np.log(1-a_out))).sum()) + reg           #formula for the regularised cost function

    return(J)



def grad_NN(nn_params,layers_data, X, Y, Lambda =0):
    """
    It compute the Backward Propagation and gradient of the Neural Network.
    The function as input
        - nn params  : a vector of weights parameters
        - layers_data: a vector containing the number of units at each layer (input and output included)
        - X : the training set matrix X
        - Y : the label vector Y
        - J : the regularisation parameter, by default is zero
   The output is a single vector containing all the unrolled gradients. This is because the fmin_cg accept
   only vectors and not matrices.
   """
    m = np.shape(X)[0]
    a_vet,z,Y,Theta_vec = ForwardProp(nn_params, layers_data, X, Y)  #computing the units values, the activated values z, the label matrix Y and the weights matrices

    delta = [0 for i in range(len(layers_data)-1)]                   # initialise the little delta list
    for i in reversed(range(1,len(layers_data))):
        if i == len(layers_data)-1:
            delta[i-1] = a_vet[-1]-Y                                # this is the value of the delta_out
        else:
            delta[i-1]= (Theta_vec[i][:,1:].T@delta[i])*sigmoidGradient(z[i-1]) #formula for computing the inner delta.

    Delta = [0 for i in range(len(layers_data)-1)]        # Initialise the big delta list. 
    for i in reversed(range(0, len(layers_data)-1)):
        Delta[i] = delta[i]@a_vet[i].T                      # The entry i is a matrix with same shape of the weights matrix i

    reg_grad = [(Lambda/m)*(np.hstack((np.zeros(np.shape(Theta)[0])[np.newaxis].T, Theta[:,1:]))) for Theta in Theta_vec] #regularisation factors
    Theta_grad =[(1/m)*Del+reg for Del,reg in zip(Delta,reg_grad)]                     # gradient matrices
    grad_vec = np.concatenate([Theta_g.reshape(Theta_g.size,order = 'F') for Theta_g in Theta_grad]) #gradients vector

    return grad_vec


def NN_fit(X, Y, hidden_layer_sizes = [2], max_iter = 50, Lambda = 0):
    """
    The function will train a Neural Network Classifier on your set X with labels in the vector Y.
    It returns a single vector that containing all the optimal weights for the classifier after max_iter iterations
    of the optimisation function fmin_cg. To use it for the prediction you need to reshape all the Weights matrix.
    The function reshape_thetas will do it for you.

    You have the following options:

           -hidden_layers_sizes (optional) : is a vector that contains the numbers of units in each internal
                                             layer of the Neural Network. By default the Neural Network has
                                             a single internal layer with two units. More entries in the vector
                                             means more layers.

           -max_iter(optional): it determines the maximum number of iterations that the function fmin_cg must
                                perform.

           -Lambda: is the regularisation parameter. By default is zero. Increase the parameter if you are 
                    experiencing high variance or reduce it if you have high bias.
    """

          #### UTILITIES ####
    input_layer_size = np.shape(X)[1]        #recovering the input units from the experimental matrix
    num_labels = len(np.unique(Y))           #recovering the number of labels from the data stored in the label vector Y
    layers_data = [input_layer_size] + hidden_layer_sizes + [num_labels]         # a vector that contains the info about the units of all the layers


            #### INITIALISATION OF THE PARAMETERS ####
    Thetas = [Rand_param_Theta(layers_data[i], layers_data[i+1]) for i in range(len(layers_data)-1)]         #random initialisation of the layers matrices Weights
    nn_params_init = np.concatenate([Theta_p.reshape(Theta_p.size,order = 'F') for Theta_p in Thetas])       #it creates a vector with all the entries of the Weights

           #### FITTING OF THE NEURAL NETWORK ####
    args = (layers_data, X, Y, Lambda)         #arguments to pass to the functions
    xopt = fmin_cg(cost_func_NN, x0 = nn_params_init, fprime = grad_NN, args = args, maxiter = max_iter)  #computes the optimal Weights vector

    return xopt

## Testing ##
We now test our implementation on a dataset provided by Andrew NG. The aim is to train the Neural Network in order to recognise hand writing. We have a $5000\times400$ matrix $X$ as training set and a $1\times5000$ label vector $Y$.

In [204]:
from scipy.io import loadmat
data = loadmat('ex4data1.mat')
layers_data = [25]
X = data['X']
Y = data['y']
Theta_opt = NN_fit(X,Y,layers_data, max_iter = 100)

         Current function value: 0.050756
         Iterations: 100
         Function evaluations: 252
         Gradient evaluations: 252


Now that we have the weights matrices we look how good the Neural Netwrok perform on the the training set

In [223]:
pred = nn_predict(Theta_opt, [400,25,10],X, 1)
Y_train = Y.reshape(Y.size)
round((pred == Y_train).mean()*100,2)

99.8

For a more realistic result we need to split our set and see what happen.

In [224]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.25, random_state=1)

In [225]:
layers_data = [25]
Theta_opt = NN_fit(X_train,Y_train,layers_data, max_iter = 100)

         Current function value: 0.027718
         Iterations: 100
         Function evaluations: 266
         Gradient evaluations: 266


In [227]:
pred = nn_predict(Theta_opt, [400,25,10],X_valid, 1)
Y_valid_aux = Y_valid.reshape(Y_valid.size)
round((pred == Y_valid_aux).mean()*100,2)

92.32

That is still a nice performance.